In [ ]:
# Requirements
!pip install nibabel vtk pyacvd pyvista simpleitk 

In [ ]:
import k3d
import numpy as np
from k3d.helpers import download
import pyvista
from vtk.util import numpy_support
import SimpleITK as sitk
import vtk

In [ ]:
filename = download('https://vedo.embl.es/examples/data/embryo.slc')
reader = vtk.vtkSLCReader()
reader.SetFileName(filename)
reader.Update()
vti = reader.GetOutput()

bounds = vti.GetBounds()
x, y, z = vti.GetDimensions()
volume_data = numpy_support.vtk_to_numpy(vti.GetPointData().GetArray(0)).reshape(-1, y, x)

In [ ]:
seg = sitk.BinaryThreshold(
    sitk.GetImageFromArray(volume_data),
    lowerThreshold=20,
    upperThreshold=255
)
mask = sitk.GetArrayFromImage(seg)

def get_bounds(img):
    origin = img.GetOrigin()
    size = np.array(img.GetSpacing()) * np.array(img.GetDimensions())

    return np.array([origin[0], origin[0] + size[0],
                   origin[1], origin[1] + size[1],
                   origin[2], origin[2] + size[2]])


def contour(data, bounds, values, clustering_factor=6):
    grid = pyvista.ImageData(dimensions=data.shape[::-1],
    spacing=(bounds[1::2] - bounds[::2]) / np.array(data.shape[::-1]),
    origin=bounds[::2])

    grid.point_data["values"] = data.flatten()
    mesh = grid.contour(values, grid.point_data["values"], method='flying_edges')

    return mesh

mesh = k3d.vtk_poly_data(contour(mask, get_bounds(vti), [1]), color=0xff0000)

In [ ]:
embryo = k3d.volume_slice(volume_data.astype(np.float16), 
                          mask=mask,
                          active_masks=[1],
                          color_map_masks = [0, 0xff0000],
                          color_map=np.array(k3d.paraview_color_maps.Grayscale, dtype=np.float32), 
                          slice_z=volume_data.shape[0]//2,
                          slice_y=volume_data.shape[1]//2,
                          slice_x=volume_data.shape[2]//2,
                          bounds=bounds)

plot = k3d.plot(camera_mode='volume_sides', grid_visible=False, background_color=0)
plot += embryo
plot += mesh
plot.slice_viewer_mask_object_ids = [mesh.id]

In [ ]:
plot.display()